*   bottleneck_support.py



In [0]:
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [0]:
input_size = 10
epochs = 2
batches = 64
lr = 0.01

In [0]:
def encoder(input_size):
    def wrapper(num):
        ret = [int(i) for i in '{0:b}'.format(num)]
        return [0] * (input_size - len(ret)) + ret
    return wrapper


def decoder(array):
    ret = 0
    for i in array:
        ret = ret * 2 + int(i)
    return ret


def training_test_gen(x, y):
    assert len(x) == len(y)
    indices = np.random.permutation(range(len(x)))
    split_size = int(0.9 * len(indices))
    trX = x[indices[:split_size]]
    trY = y[indices[:split_size]]
    teX = x[indices[split_size:]]
    teY = y[indices[split_size:]]
    return trX, trY, teX, teY


def get_data(input_size):
    x = []
    y = []
    binary_enc = encoder(input_size)
    for i in range(1000):
        x.append(binary_enc(i))
        if i % 15 == 0:
            y.append([1, 0, 0, 0])
        elif i % 5 == 0:
            y.append([0, 1, 0, 0])
        elif i % 3 == 0:
            y.append([0, 0, 1, 0])
        else:
            y.append([0, 0, 0, 1])
    return training_test_gen(np.array(x), np.array(y))


def check_fizbuz(i):
    if i % 15 == 0:
        return 'fizbuz'
    elif i % 5 == 0:
        return 'buz'
    elif i % 3 == 0:
        return 'fiz'
    else:
        return 'number'

In [0]:
class FizBuzNet(nn.Module):

    def __init__(self, input_size, output_size):
        super(FizBuzNet, self).__init__()
        # A simple heuristic to find the hiddenlayer size
        hidden_size = 100
        self.hidden = nn.Linear(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, batch):
        hidden = self.hidden(batch)
        activated = F.sigmoid(hidden)
        out = self.out(activated)
        return F.sigmoid(out)

In [0]:
trX, trY, teX, teY = get_data(input_size)
if torch.cuda.is_available():
    dtype = torch.cuda.FloatTensor
else:
    dtype = torch.FloatTensor
x = torch.from_numpy(trX).type(dtype)
y = torch.from_numpy(trY).type(dtype)

net = FizBuzNet(input_size, 4)
net = net.to(device)
loss_fn = nn.MSELoss()
optimizer = optim.Adam(net.parameters(), lr=lr)
x_ = x[0:10]
y_ = y[0:10]

with torch.autograd.profiler.profile() as prof:
    hyp = net(x_)

print(prof)
prof.export_chrome_trace('chrometrace')
print(prof.key_averages())
print(prof.table('cpu_time'))


loss = loss_fn(hyp, y_)
loss.backward()
optimizer.step()

------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  -----------------------------------  
Name                Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     CUDA total %     CUDA total       CUDA time avg    Number of Calls  Input Shapes                         
------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  -----------------------------------  
unsigned short      7.30%            28.984us         7.30%            28.984us         28.984us         NaN              0.000us          0.000us          1                []                                   
addmm               45.00%           178.746us        45.00%           178.746us        178.746us        NaN              0.000us          0.000us         

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


*   otherenv.py

In [0]:
from scipy.signal import convolve2d, correlate2d
from torch.nn.modules.module import Module
from torch.nn.parameter import Parameter
import torch
from torch.autograd import Function
from numpy.fft import rfft2, irfft2

In [0]:
## 파라미터가 없는 신경 네트워크 레이어
## NumPy 호출
class BadFFTFunction(Function):

    def forward(self, input):
        numpy_input = input.detach().numpy()
        result = abs(rfft2(numpy_input))
        return input.new(result)

    def backward(self, grad_output):
        numpy_go = grad_output.numpy()
        result = irfft2(numpy_go)
        return grad_output.new(result)

In [0]:
## 어떠한 파라미터도 가지고 있지 않기 때문에, nn.Module class가 아닌 단순히 함수로 선언 가능
def incorrect_fft(input):
    return BadFFTFunction()(input)

# 생성된 계층 사용
input = torch.randn(8, 8, requires_grad=True)
result = incorrect_fft(input)
print(result)
result.backward(torch.randn(result.size()))
print(input)

tensor([[ 0.7769, 12.4145,  6.1387,  6.8268, 12.2997],
        [ 3.5703,  5.9944,  7.1139,  7.9856, 10.8436],
        [ 9.0111,  9.9996, 15.4300,  5.8045,  1.6839],
        [ 8.8676,  4.6452,  4.1934, 15.6487,  2.6560],
        [10.6404,  8.1674,  1.4506,  8.1888,  5.3366],
        [ 8.8676,  5.1615,  2.7822,  9.4244,  2.6560],
        [ 9.0111,  6.2403,  6.8715,  6.8352,  1.6839],
        [ 3.5703, 14.0815,  9.7259,  2.1732, 10.8436]],
       grad_fn=<BadFFTFunction>)
tensor([[-0.9711, -0.9519,  0.6763,  0.9975,  0.6966,  1.1866,  0.6841,  0.2102],
        [-0.3545,  1.2948, -0.5142, -0.5564, -0.6927,  0.8253,  2.1920,  0.6857],
        [ 1.1669, -0.3851, -0.5955, -1.0725, -2.2673,  1.2097,  1.7063, -1.4818],
        [ 0.6880,  0.3396,  0.5218, -1.5066,  0.5540, -0.4042, -0.6406,  1.0090],
        [ 1.3870, -3.2352, -0.1160,  0.1354, -0.1221, -0.7590, -0.1295, -0.7277],
        [ 1.5963,  0.5002,  1.5813, -0.6002,  0.0748, -0.3494,  0.5254,  0.9195],
        [-0.5071,  0.2542, -1.0777

/pytorch/torch/csrc/autograd/python_function.cpp:622: UserWarning: Legacy autograd function with non-static forward method is deprecated and will be removed in 1.3. Please use new-style autograd function with static forward method. (Example: https://pytorch.org/docs/stable/autograd.html#torch.autograd.Function)


In [0]:
## 학습 가능한 가중치를 갖는 신경 네트워크 계층 생성
## SciPy 호출
class ScipyConv2dFunction(Function):
    @staticmethod
    def forward(ctx, input, filter):
        # NumPy에 cast 할 수 있도록 분리
        input, filter = input.detach(), filter.detach()
        result = correlate2d(input.numpy(), filter.detach().numpy(), mode='valid')
        ctx.save_for_backward(input, filter)
        return input.new(result)

    @staticmethod
    def backward(ctx, grad_output):
        grad_output = grad_output.detach()
        input, filter = ctx.saved_tensors
        grad_input = convolve2d(grad_output.numpy(), filter.t().numpy(), mode='full')
        grad_filter = convolve2d(input.numpy(), grad_output.numpy(), mode='valid')

        return grad_output.new_tensor(grad_input), grad_output.new_tensor(grad_filter)

In [0]:
class ScipyConv2d(Module):

    def __init__(self, kh, kw):
        super(ScipyConv2d, self).__init__()
        self.filter = Parameter(torch.randn(kh, kw))

    def forward(self, input):
        return ScipyConv2dFunction.apply(input, self.filter)

In [0]:
module = ScipyConv2d(3, 3)
print(list(module.parameters()))
input = torch.randn(10, 10, requires_grad=True)
output = module(input)
print(output)
output.backward(torch.randn(8, 8))
print(input.grad)

[Parameter containing:
tensor([[-1.1066, -0.6778,  1.7192],
        [-1.5343, -0.9657, -0.8305],
        [-0.1279,  0.4434, -2.3637]], requires_grad=True)]
tensor([[ 5.2346,  0.9000,  5.1764, -6.8713,  0.6142,  6.1871,  0.9441, -4.8945],
        [-1.9496, -0.0778, -0.4449,  2.5229, -2.5123,  3.4632,  3.8921,  1.8022],
        [-1.3675,  0.5252, -6.2152,  2.1431, -0.0800,  0.1739,  1.4143,  0.5641],
        [ 0.3921, -0.4304, -5.4476, -3.6244, -0.5503,  3.7724, -1.9953, -4.8017],
        [-1.5782, -1.3202,  2.5477, -2.1432, -7.4357,  5.5934, -0.1721, -4.7424],
        [ 0.6395,  7.4393,  1.6977, -0.8717, -4.7625, -1.0466,  2.2804,  1.5283],
        [-1.7883, -0.2413, -4.7623,  1.7718,  0.5406, -8.3984,  2.9529,  5.1917],
        [ 1.2366, -4.8144, -2.1437,  1.2044, -3.1919,  2.3996, -2.4476,  6.8843]],
       grad_fn=<ScipyConv2dFunctionBackward>)
tensor([[-1.0409e+00, -1.5644e+00,  8.7278e-01,  2.3866e+00,  1.2032e+00,
         -1.3906e+00, -1.7672e+00, -3.6139e-02, -2.4886e-01, -2.303

*   profile_support.py

In [0]:
import time
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np

In [0]:
input_size = 10
epochs = 2
batches = 64
lr = 0.01

In [0]:
def encoder(input_size):
    def wrapper(num):
        ret = [int(i) for i in '{0:b}'.format(num)]
        return [0] * (input_size - len(ret)) + ret
    return wrapper

In [0]:
def decoder(array):
    ret = 0
    for i in array:
        ret = ret * 2 + int(i)
    return ret

In [0]:
def training_test_gen(x, y):
    assert len(x) == len(y)
    indices = np.random.permutation(range(len(x)))
    split_size = int(0.9 * len(indices))
    trX = x[indices[:split_size]]
    trY = y[indices[:split_size]]
    teX = x[indices[split_size:]]
    teY = y[indices[split_size:]]
    return trX, trY, teX, teY

In [0]:
def get_data(input_size):
    x = []
    y = []
    binary_enc = encoder(input_size)
    for i in range(1000):
        x.append(binary_enc(i))
        if i % 15 == 0:
            y.append([1, 0, 0, 0])
        elif i % 5 == 0:
            y.append([0, 1, 0, 0])
        elif i % 3 == 0:
            y.append([0, 0, 1, 0])
        else:
            y.append([0, 0, 0, 1])
    return training_test_gen(np.array(x), np.array(y))

In [0]:
def check_fizbuz(i):
    if i % 15 == 0:
        return 'fizbuz'
    elif i % 5 == 0:
        return 'buz'
    elif i % 3 == 0:
        return 'fiz'
    else:
        return 'number'

In [0]:
class FizBuzNet(nn.Module):

    def __init__(self, input_size, output_size):
        super(FizBuzNet, self).__init__()
        # A simple heuristic to find the hiddenlayer size
        hidden_size = 100
        self.hidden = nn.Linear(input_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

    def forward(self, batch):
        hidden = self.hidden(batch)
        activated = F.sigmoid(hidden)
        out = self.out(activated)
        return F.sigmoid(out)

In [0]:
trX, trY, teX, teY = get_data(input_size)
if torch.cuda.is_available():
    dtype = torch.cuda.FloatTensor
else:
    dtype = torch.FloatTensor
x = torch.from_numpy(trX).type(dtype)
y = torch.from_numpy(trY).type(dtype)

net = FizBuzNet(input_size, 4)
loss_fn = nn.MSELoss()
net = net.to(device)
optimizer = optim.Adam(net.parameters(), lr=lr)
x_ = x[0:10]
y_ = y[0:10]
hyp = net(x_)

print(prof)
prof.export_chrome_trace('chrometrace')
print(prof.key_averages())
print(prof.table('cpu_time'))

loss = loss_fn(hyp, y_)
loss.backward()
optimizer.step()

------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  -----------------------------------  
Name                Self CPU total %  Self CPU total   CPU total %      CPU total        CPU time avg     CUDA total %     CUDA total       CUDA time avg    Number of Calls  Input Shapes                         
------------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  ---------------  -----------------------------------  
unsigned short      7.30%            28.984us         7.30%            28.984us         28.984us         NaN              0.000us          0.000us          1                []                                   
addmm               45.00%           178.746us        45.00%           178.746us        178.746us        NaN              0.000us          0.000us         

/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1351: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
